## Creating information gain metric structure

## making structure where infromation gain is easily accessed

In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.cluster import KMeans
from ontobio.ontol_factory import OntologyFactory
from ontobio.assoc_factory import AssociationSetFactory
import os
import json
import math

In [2]:
FOLDER = "./../data/"

In [3]:
namespace_list = ["cellular_component", "molecular_function", "biological_process"]

#maybe also save the alternative IDs, not sure right now
class Node:
  def __init__(self, name):
    self.name = name
    self.visited = False  # for BFS
    self.reachable_nodes_count = 1  
    self.infromationMetric = 0
    self.namespace = -1
    #without alternative IDs
    self.parents = []  
    self.children = []
    


For now only checking part_of and is_a relationships. not regulates, positively_regulates and negatively_regulates

In [4]:
goFile = open(FOLDER + 'go-basic.obo', 'r')
Lines = goFile.readlines()

nodeDict = dict()
alternative_ID_dict = dict()

count_IS_a_relationship = 0
count_part_of_relationship = 0
lastID  = ""
altID = []



for line in Lines:
    #NEW GO term
    if line[0:4] == "id: ":
        if altID:
            alternative_ID_dict[lastID] = altID
        altID = [] 
        
        lastID = line[4:-1]
        if not (lastID in nodeDict):
            nodeDict[lastID] = Node(lastID)
    #when the lastID was checked to be obsolete, then move on, until there is new lastID
    if lastID == "obsolete":
        continue
    
    #adding namespace

    if line[0:10] == "namespace:":
        namespace = line.split()[1]
        for i in range(len(namespace_list)):
            if (namespace == namespace_list[i]):
                nodeDict[lastID].namespace = i
                
    #ALternative name for ID
    if line[0:8] == "alt_id: ":
        altID.append(line[8:-1])
    
    #Check if gene is obsolete
    if line[0:11] == "is_obsolete" :
        altID = []
        nodeDict.pop(lastID)
        lastID = "obsolete"
        
    #IS_a relationship    
    if line[0:5] == "is_a:":
        count_IS_a_relationship +=1
        isA = line[5:-1].split()[0]
        nodeDict[lastID].parents.append(isA)  
        if not (isA in nodeDict):
            nodeDict[isA] = Node(isA)
        nodeDict[isA].children.append(lastID)
        
    # part_of relationship    
    if line[0:21] == "relationship: part_of":
        count_part_of_relationship +=1
        partOf = line[21:-1].split()[0]
        nodeDict[lastID].parents.append(partOf)  
        if not (partOf in nodeDict):
            nodeDict[partOf] = Node(partOf)
        nodeDict[partOf].children.append(lastID)
        
    
goFile.close()        

alternative_ID_dict[lastID] = altID
print(count_IS_a_relationship)
print(count_part_of_relationship)        


68896
6815


In [5]:
#simple test if there is same number of parents as childrens
count_children = 0
count_parents = 0
for ID, node in nodeDict.items():
    if node.children:
        count_children += len(node.children)
    if node.parents:
        count_parents += len(node.parents)

print(count_children == count_parents)
print(count_IS_a_relationship + count_part_of_relationship == count_parents)

True
True


In [6]:
#Removing regulates, negatively_regulates,positively_regulates, part_of, term_tracker_item
nodes_to_remove = ["regulates", "negatively_regulates", "positively_regulates", "part_of", "term_tracker_item"]
for node in nodes_to_remove:
    nodeDict.pop(node)

In [7]:
#creating root element
root = Node("root")
for ID, node in nodeDict.items():
    if not node.parents:
        root.children.append(node.name)
        node.parents.append(root.name)
        
nodeDict["root"] = root
numberOfNodes = len(nodeDict)

In [8]:
# adding to the dictionary alternative ID, these points to the same node as the their original ID
count_Alt_ID = 0
for ID, alt_ID_list in alternative_ID_dict.items():
    for alt_ID in alt_ID_list:
        if (alt_ID in nodeDict):
            count_Alt_ID += 1
            
        nodeDict[alt_ID] = nodeDict[ID]
if count_Alt_ID !=0:
    print("nieco je zle") # TEST if the alt_ID is already in nodeDict, if it were, my proccesing of GO would be bad

In [9]:
# check visited and change it before usage of BFS
def BFS(nodeID, nodeDict):
    current_node = nodeDict[nodeID]
    for child in current_node.children:
        if not nodeDict[child].visited:
            nodeDict[child].visited = True
            BFS(child, nodeDict)
            current_node.reachable_nodes_count += nodeDict[child].reachable_nodes_count

BFS("root", nodeDict) 
#test if bfs is valid
root.reachable_nodes_count == numberOfNodes

True

In [10]:
for node in nodeDict.values():
    node.infromationMetric = node.reachable_nodes_count / numberOfNodes

In [11]:
#just test
for i  in root.children:
    print(nodeDict[i].infromationMetric)

0.6448461502761406
0.26131247969554927
0.09381816494175524


## Clustering and metric

### getting data for clustering

In [12]:
gene_disease_csv = pd.read_csv(FOLDER+ "gene_disease_view.csv")
gene_disease_series = gene_disease_csv.groupby(["geneName"])["diseaseId"].apply(list)
gene_disease_dict = gene_disease_series.to_dict()

gene_to_list_of_diseases = []

#propably could be improved somehow with pandas functionality
for gene_name, disease_ids in gene_disease_dict.items():
    new_dict = {'diseaseId': disease_ids, 'geneName': str(gene_name)}
    gene_to_list_of_diseases.append(new_dict)

C:\Users\samue\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Creating ontology

In [17]:
HUMAN = 'NCBITaxon:9606'
PART_OF = 'BFO:0000050'

ofactory = OntologyFactory()
ont = ofactory.create("GO").subontology(relations=['subClassOf', PART_OF])

afactory = AssociationSetFactory()
aset = afactory.create(ontology=ont,
                       subject_category='gene',
                       object_category='function',
                       taxon=HUMAN)

2023-05-15 12:30:42 [WARNING] [PID:5948 TID:9364] [golr_query.py:1804 in `translate_docs_compact`]  >1 relation: ['not', 'contributes_to']
2023-05-15 12:30:42 [WARNING] [PID:5948 TID:9364] [golr_query.py:1804 in `translate_docs_compact`]  >1 relation: ['not', 'colocalizes_with']
2023-05-15 12:30:42 [WARNING] [PID:5948 TID:9364] [golr_query.py:1804 in `translate_docs_compact`]  >1 relation: ['not', 'colocalizes_with']
2023-05-15 12:30:42 [WARNING] [PID:5948 TID:9364] [golr_query.py:1804 in `translate_docs_compact`]  >1 relation: ['not', 'colocalizes_with']
2023-05-15 12:30:42 [WARNING] [PID:5948 TID:9364] [golr_query.py:1804 in `translate_docs_compact`]  >1 relation: ['not', 'contributes_to']
2023-05-15 12:30:43 [WARNING] [PID:5948 TID:9364] [golr_query.py:1804 in `translate_docs_compact`]  >1 relation: ['not', 'colocalizes_with']
2023-05-15 12:30:43 [WARNING] [PID:5948 TID:9364] [golr_query.py:1804 in `translate_docs_compact`]  >1 relation: ['not', 'contributes_to']
2023-05-15 12:30:43

### Metric definition

In [26]:
def union(lst1, lst2):
    return list(set(lst1) | set(lst2))

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

In [20]:
def metric_based_on_information_theory(cluster):
    counter = 0
    result = 0
    size_of_cluster = len(cluster)
    for i in range(size_of_cluster):
        for j in range(i + 1, size_of_cluster):
            goTerms = intersection(ont.ancestors(cluster[i]),ont.ancestors(cluster[j]))
            minimum_infromation_metric = 1
            for goTerm in goTerms:
                infromation_metric = nodeDict[goTerm].infromationMetric
                if minimum_infromation_metric > infromation_metric:
                    minimum_infromation_metric = infromation_metric
                    
                
            result -= math.log(minimum_infromation_metric)
            counter +=1
    if counter == 0 :
        return 0
    return result/counter
            

In [29]:
def jaccard_similarity(gene1, gene2):
    ancestors1 = ont.ancestors(gene1)
    ancestors2 = ont.ancestors(gene2)
    union_genes = union(ancestors1,ancestors2)
    num_union = len(union_genes)
    if  num_union == 0:
        return 0
    return len(intersection(ancestors1,ancestors2)) / num_union

def metric_cluster_jaccard(cluster):
    counter = 0
    result = 0
    size_of_cluster = len(cluster)
    for i in range(size_of_cluster):
        for j in range(i + 1, size_of_cluster):
            result += jaccard_similarity(cluster[i],cluster[j])
            counter +=1
    if counter == 0 :
        return 0
    return result/counter
            

### Feature extraction and simple clustering

In [14]:
vec = DictVectorizer()
matrixGeneDisease = vec.fit_transform(gene_to_list_of_diseases)
matrixGeneDisease

cluster = KMeans()
result = cluster.fit(matrixGeneDisease)
result.labels_

array([3, 3, 3, ..., 6, 3, 3])

### Creating arrays of geneIdentificators which ont takes , for each cluster. (information metric)

In [13]:
#geneName to GO
f = open(FOLDER + 'mapping_gene_to_go.json')
mapping_geneName_to_GO = json.load(f)
f.close()

### Mapping geneName to GO terms

In [15]:
#Could be added before feature extraction, but this is not computionaly expensive, and it is easier to see which genes are in which clusters, 
#otherwise only GO terms would be seen
clustersGO = []
for i in range(max(result.labels_) + 1):
    clustersGO.append([])

for i in range(len(result.labels_)):
    if gene_to_list_of_diseases[i]["geneName"] in mapping_geneName_to_GO:
        clustersGO[result.labels_[i]].append(mapping_geneName_to_GO[gene_to_list_of_diseases[i]["geneName"]])

### Calculating metric values for clusters

In [30]:
count = 0
for i in clustersGO:
    if len(i) >= 1000:
        continue
    print("Information metric: "  + str(metric_based_on_information_theory(i)) )
    print("Jaccard similarity metric: " + str(metric_cluster_jaccard(i)) )
    

Information metric: 0
Jaccard similarity metric: 0
Information metric: 1.1741925972478595
Jaccard similarity metric: 0.05779025449919795
Information metric: 8.368553962270132
Jaccard similarity metric: 0.391304347826087
Information metric: 2.451797574325175
Jaccard similarity metric: 0.08898046398046398
Information metric: 1.2325186914849864
Jaccard similarity metric: 0.06846816482300143
